# Run Perdido

In [ ]:
import os
from perdido.geoparser import Geoparser
from tqdm import tqdm
from lxml import etree

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path ='/Users/lmoncla/Documents/Data/Corpus/Choucas'
output_path = os.path.join(path ,'Perdido')

In [ ]:
def text_preprocessing(text):
    text = text.replace(' ', ' ')
    text = text.replace('œ', 'oe')
    return text.replace('\n', ' ').strip()

def run_perdido(text, output_dir, filename, geoparser):
    try :
        doc = geoparser(text_preprocessing(text))
    except etree.XMLSyntaxError as e:
        return None
    
    try:
        os.mkdir(os.path.join(output_dir, filename))
    except OSError:
        pass
    try:
        doc.to_xml(os.path.join(output_dir, filename, filename + '.xml'))
        #doc.to_iob(os.path.join(output_dir, 'T'+str(volume), filepath_name + '.tsv'))

        iob = 'n\tform\tlemma\tpos\ttype\n'
        for token in doc:
                iob += token.tsv_format() + '\n'

        with open(os.path.join(output_dir, filename, filename + '.tsv'), 'w') as f:
            f.write(iob)

        doc.to_html(os.path.join(output_dir, filename, filename + '_ent.html'), 'ent')
        doc.to_html(os.path.join(output_dir, filename, filename + '_span.html'), 'span')
    except FileNotFoundError as e:
        print('Error', filename)
        print(e)

    return len(doc.ne_place), len(doc.ne_person)


In [ ]:
geoparser = Geoparser(version='Standard', sources=['wiki_gaz'])

In [ ]:
for doc in tqdm(sorted(os.listdir(path))):
    try:
        num = int(doc.split('_')[-1])
        #print(num)
        filename = 'FichierNettoye_' + str(num)
        if os.path.exists(os.path.join(output_path, filename, filename + '.xml')):
            continue

        with open(os.path.join(path, doc, filename), 'r') as f:
            text = f.read()

            run_perdido(text, output_path, filename, geoparser)

    except ValueError:
        pass

In [ ]:
path = '/Users/lmoncla/Documents/Data/Corpus/Visorando/txt'
output_path = '/Users/lmoncla/Documents/Data/Corpus/Visorando/Perdido'

In [ ]:

def run_perdido2(text, output_dir, filename, geoparser):
    try :
        doc = geoparser(text_preprocessing(text))
    except etree.XMLSyntaxError as e:
        return None
    try:
        doc.to_xml(os.path.join(output_dir, filename + '.xml'))
    except FileNotFoundError as e:
        print('Error', filename)
        print(e)

    return len(doc.ne_place), len(doc.ne_person)

In [ ]:
for doc in tqdm(sorted(os.listdir(path))):
    if doc.endswith('.txt'):

        filename = doc[:-4]
        if os.path.exists(os.path.join(output_path, filename + '.xml')):
            continue

        with open(os.path.join(path, doc), 'r') as f:
            text = f.read()
            try:
                run_perdido2(text, output_path, filename, geoparser)
            except:
                print(filename)
